# 모델 개발

## 데이터 불러오기 

In [91]:
# 구글 드라이브를 /gdrive에 마운트 합니다.
from google.colab import drive
#drive.flush_and_unmount()  # 파일목록 갱신이 안될때는 이 부분 주석처리를 해제하고 실행하시기 바랍니다.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [92]:
import pandas as pd
# /gdrive/MyDrive/Lpay

data1 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_01_DEMO.csv')
data2 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_02_PDDE.csv')
data3 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_03_COP_U.csv')
data4 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_04_PD_CLAC.csv')
data5 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_05_BR.csv')
data6 = pd.read_csv('/gdrive/MyDrive/Lpay/LPOINT_BIG_COMP_06_LPAY.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 데이터 합병(LPOINT_BIG_COMP_01_DEMO + LPOINT_BIG_COMP_02_PDDE )
DEMO 데이터와 상품구매 정보 데이터를 합병 시켜 상품구매정보+연령+성별 데이터 생성

In [93]:
# Merging Data
merge_data = pd.merge(data1,data2)
merge_data.head()

,cust,ma_fem_dv,ages,zon_hlv,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M000034966,여성,40대,Z07,A01016688590,1,A01,A010045,PD0777,20210116,18,59000.0,1
1,M000034966,여성,40대,Z07,A01016688590,1,A01,A010045,PD0777,20210116,18,19000.0,1
2,M000034966,여성,40대,Z07,A01016688590,1,A01,A010045,PD0796,20210116,18,100.0,1
3,M000034966,여성,40대,Z07,A01016718167,1,A01,A010045,PD0816,20210116,18,62100.0,1
4,M000034966,여성,40대,Z07,A02357398521,1,A02,A020116,PD0630,20210325,13,19900.0,1


In [94]:
# Data Save 
# merge_data.to_csv('/gdrive/MyDrive/Lpay/merge_data.csv',index=False)

In [95]:
print("DEMO 데이터와 상품구매 데이터를 합한 총 데이터 수는: ",len(merge_data), '입니다')

DEMO 데이터와 상품구매 데이터를 합한 총 데이터 수는:  4381743 입니다


## 데이터 전처리
상품 대분류 데이터 전처리 & 데이터 클래스 딕셔너리화 및 맵핑 

In [96]:
# Drop Data 
drop_data = merge_data.drop(['cust','rct_no','buy_ct','de_hr','br_c'], axis=1)
drop_data.head(3)

,ma_fem_dv,ages,zon_hlv,chnl_dv,cop_c,pd_c,de_dt,buy_am
0,여성,40대,Z07,1,A01,PD0777,20210116,59000.0
1,여성,40대,Z07,1,A01,PD0777,20210116,19000.0
2,여성,40대,Z07,1,A01,PD0796,20210116,100.0


### 상품 대분류 데이터 전처리

In [97]:
# 상품 분류정보중 상품 대분류 데이터
clac_hlv_nm_60 = data4.clac_hlv_nm.value_counts().keys()
clac_hlv_nm_60

Index(['패션잡화', '축산물', '수산물', '스포츠패션', '문구/사무용품', '채소', '유아동의류', '과일',
       '속옷/양말/홈웨어', '조미료', '화장품/뷰티케어', '생활/주방가전', '퍼스널케어', '원예/애완', '구기/필드스포츠',
       '조리식품', '식기/조리기구', '과자', '침구/수예', '자동차용품', '아웃도어/레저', '건강식품', '출산/육아용품',
       '냉장식품', '주방잡화', '가구', '완구', '건해산물', '청소/세탁/욕실용품', '여성의류', '대용식',
       '세제/위생', '영상/음향가전', '남성의류', '건강용품', '음료', '테넌트/음식점', '주류', '시즌스포츠',
       '인테리어/조명', '커피/차', '냉동식품', '헬스/피트니스', '양곡', '서적/음반/악기', '공구/안전용품',
       '계절가전', '컴퓨터', '병통조림', '생활/렌탈서비스', '냉장/세탁가전', '상품권', '유제품', '기타(비상품)',
       '담배', '여행/레저서비스', '모바일', '유아식품', '기타상품', '금융/보험서비스'],
      dtype='object')

In [98]:
# 데이터 분석을 통한 데이터를 토대로 제외 품목 선정
# ('패션잡화', 0), ('축산물', 1), ('수산물', 2), ('채소', 5), ('과일', 6)  , ('과자', 16)

In [99]:
# 상품정보 -> 상품 대분류를 위한 합병
merge_2 = pd.merge(drop_data,data4)
merge_2.zon_hlv.value_counts().keys()

Index(['Z17', 'Z10', 'Z16', 'Z11', 'Z04', 'Z06', 'Z13', 'Z05', 'Z08', 'Z09',
       'Z15', 'Z14', 'Z12', 'Z03', 'Z01', 'Z07', 'Z02'],
      dtype='object')

### 데이터 클래스 딕셔너리화

In [100]:
# 상품분류 클래스 딕셔너리화
# class 1923 -> 60 
pd_dict = {}
i = 0 
for nm in clac_hlv_nm_60:
  pd_dict[nm] = i
  i += 1
# 지역 데이터 -> 정수화 딕셔너리
zon_name = merge_2.zon_hlv.value_counts().keys()
zon_dict = {}
i = 0 
for nm in zon_name:
  zon_dict[nm] = i
  i += 1 

In [101]:
def pd_cls(numbers):
  return pd_dict[numbers]
def date_cls(dd): # 공휴일 데이터 
  for d in [20210101 ,20210211,20210212,20210213,20210301,20210505,20210519,20210606,20210815,20210920,20210921,20210922,20211003,20211009,20211225]:
    if d==dd:
      return 1
    else:
      return 0
def zon_cls(zonn):
  return zon_dict[zonn]
def target_y(data):
  if data == '여성70대' or data == '여성60대'or data == '남성60대'or data == '남성70대':
    return 1
  else:
    return 0

In [102]:
merge_2['mfages'] = merge_2['ma_fem_dv']+merge_2['ages']
merge_2['zon_hlv']  = merge_2['zon_hlv'].map(lambda x : zon_cls(x))
merge_2['de_dt']  = merge_2['de_dt'].map(lambda x : date_cls(x))
merge_2['clac_hlv_num']  = merge_2['clac_hlv_nm'].map(lambda x : pd_cls(x))
merge_2['labels']  = merge_2['mfages'].map(lambda x : target_y(x))
merge_2 = merge_2.drop(['clac_hlv_nm','pd_c','cop_c','ma_fem_dv','ages','mfages'],axis=1)

In [103]:
# 제외 품목 적용
merge_2 = merge_2[(merge_2.clac_hlv_num != 0) & (merge_2.clac_hlv_num != 1) & (merge_2.clac_hlv_num != 2) & (merge_2.clac_hlv_num != 5) & (merge_2.clac_hlv_num != 6)]

In [104]:
# 독립변수 & 종속변수
merge_y = merge_2['labels']
merge_x = merge_2.drop(['labels','pd_nm','clac_mcls_nm'],axis=1)

In [105]:
# 데이터 확인
merge_x

,zon_hlv,chnl_dv,de_dt,buy_am,clac_hlv_num
0,15,1,0,59000.0,3
1,15,1,0,19000.0,3
2,3,1,0,239000.0,3
3,0,1,0,179000.0,3
4,0,1,0,3000.0,3
...,...,...,...,...,...
4381738,2,1,0,139000.0,32
4381739,2,2,0,199000.0,20
4381740,9,2,0,187050.0,20
4381741,1,1,0,800000.0,13


In [ ]:
# 종속변수 비율 확인
merge_y.value_counts()/len(merge_y)

0    0.925161
1    0.074839
Name: labels, dtype: float64

## 데이터 불균형 문제 해결
SMOTE 기법을 사용하여 데이터 불균형 문제 해결

In [106]:
from imblearn.over_sampling import SMOTE 
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

# SMOTE 모델 사용
X_samp, y_samp = SMOTE(random_state=4).fit_resample(merge_x, merge_y)

### 종속변수 데이터 균형 확인

In [107]:
y_samp.value_counts()/len(y_samp)

0    0.5
1    0.5
Name: labels, dtype: float64

### 데이터 분리

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_samp, y_samp, test_size=0.2, random_state=42,stratify=y_samp)

In [109]:
y_train.value_counts()/len(merge_y)

0    0.740128
1    0.740128
Name: labels, dtype: float64

## 모델 선정
랜덤 포레스트 모델 사용

### 모델 생성

In [110]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
clf = RandomForestClassifier(
            n_estimators=25, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            criterion='entropy', ## 불순도 측도
            max_depth=10, ## 개별 나무의 최대 깊이
            max_features='sqrt', ## 매 분리시 랜덤으로 뽑을 변수 개수
            max_samples=1.0, ## 붓스트랩 샘플 비율 => 1이면 학습데이터를 모두 샘플링한다.
            bootstrap=True, ## 복원 추출,  False이면 비복원 추출
            oob_score=True, ## Out-of-bag 데이터를 이용한 성능 계산
            random_state=151
        )

In [111]:
feature_names = merge_x.columns
feature_names

Index(['zon_hlv', 'chnl_dv', 'de_dt', 'buy_am', 'clac_hlv_num'], dtype='object')

In [112]:
feature_names = ['zon_hlv', 'chnl_dv','de_dt' ,'buy_am', 'clac_hlv_num']
target = ['class1','class0']

### 모델 학습

In [113]:
clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


RandomForestClassifier(criterion='entropy', max_depth=10, max_features='sqrt',
                       max_samples=1.0, n_estimators=25, oob_score=True,
                       random_state=151)

### 모델 시각화

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(75, 50))
plot_tree(clf, 
          feature_names=feature_names,
          class_names=target, 
          filled=True, impurity=True, 
          rounded=True)
plt.savefig('tree_high_dpi', dpi=100)

### 모델 검증

In [115]:
clf.score(X_train,y_train)

0.6134408664616049

### 모델 테스트

In [116]:
clf.score(X_test,y_test)

0.6135075370857601

### 모델저장


In [117]:
import joblib
joblib.dump(clf, "random_forest.joblib")

['random_forest.joblib']

### 모델 불러오기

In [118]:
loaded_rf = joblib.load("./random_forest.joblib")

### 모델 예측

15지역에서 온라인으로 구매했으며 구매금액은 59000원이고 상품정보 클래스는 3인 사람이 시니어인지 예측

In [ ]:
# zon_hlv,	chnl_dv,	de_dt	,buy_am,	clac_hlv_num
import numpy as np

testData = np.array([15,1,0,59000.0,3])
testData = testData.reshape(1,-1)
pred = loaded_rf.predict(testData)
if pred==0:
  print("시니어가 아닙니다.")
